In [2]:
import os

In [3]:
%pwd

'c:\\Users\\Omar\\Desktop\\Omar_Files\\Python_Analysis\\EndToEndMachineLearningProjectAutoMobilePrice\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\Omar\\Desktop\\Omar_Files\\Python_Analysis\\EndToEndMachineLearningProjectAutoMobilePrice'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir:Path
    trained_model_file_path:Path
    updated_base_model_path:Path
    training_data:Path
    testing_data:Path
    subsample:float
    n_estimators:int
    min_samples_split:int
    min_samples_leaf:int
    max_depth:int
    learning_rate:float

In [7]:
from src.AutoMobilePriceRegression.constants import *
from src.AutoMobilePriceRegression.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        params=self.params.GradientBoostingRegressor
        prepare_base_model=self.config.prepare_base_model
        training_data=os.path.join(self.config.data_ingestion.train_data_path,"train_data.csv")
        testing_data=os.path.join(self.config.data_ingestion.test_data_path,"test_data.csv")
        

        create_directories([training.root_dir])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_file_path=Path(training.trained_model_file_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            testing_data=Path(testing_data),
            subsample=params.SUBSAMPLE,
            n_estimators=params.N_ESTIMATORS,
            min_samples_split=params.MIN_SAMPLES_SPLIT,
            min_samples_leaf=params.MIN_SAMPLES_LEAF,
            max_depth=params.MAX_DEPTH,
            learning_rate=params.LEARNING_RATE
    

        )

        return training_config

In [9]:
import os
import urllib.request as request
from sklearn.ensemble import GradientBoostingRegressor
import time
from sklearn.metrics import r2_score
from sklearn.metrics import r2_score,mean_absolute_percentage_error
#from PriceRegression.utils.common import evaluate_models
import pickle
import pandas as pd
from numpy import loadtxt

In [10]:
class Training:
    def __init__(self,config:TrainingConfig):
        self.config= config

    def initiate_Training(self):   
        
        input_feature_train_arr=loadtxt('artifacts/prepare_base_model/train_data', delimiter=',')
        input_feature_test_arr=loadtxt('artifacts/prepare_base_model/test_data', delimiter=',')
        x_train,y_train,x_test,y_test=(
            input_feature_train_arr[:,:-1],
            input_feature_train_arr[:,-1],
            input_feature_test_arr[:,:-1],
            input_feature_test_arr[:,-1]
        )
        model=GradientBoostingRegressor(
                subsample=self.config.subsample,
                n_estimators=self.config.n_estimators,
                min_samples_split=self.config.min_samples_split,
                min_samples_leaf=self.config.min_samples_leaf,
                max_depth=self.config.max_depth,
                learning_rate=self.config.learning_rate
        )
        model.fit(x_train,y_train)

        import pickle
        with open ("./artifacts/final_model.pkl","wb") as f:
           pickle.dump(model,f)

In [11]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.initiate_Training()
except Exception as e:
    raise e

[2024-07-05 17:50:30,724: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-05 17:50:30,729: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-05 17:50:30,732: INFO: common: created directory at: artifacts]
[2024-07-05 17:50:30,735: INFO: common: created directory at: artifacts/training]
